# 深度学习中的矩阵运算统一总结

（Linear / MLP / CNN / RNN 对比）

本总结统一解释不同网络结构中矩阵乘法的本质，包括：

* 为什么线性回归不需要滑动窗口
* 为什么 CNN 需要 `corr2d`
* 为什么 RNN 可以把 `[T, bs, d_in]` 看成 T 次 `[bs, d_in]`
* 各网络的本质都是“对很多向量重复使用同一个 W”

---

# 1. 线性回归 / MLP：一次性矩阵乘法

**输入：**

```
X : [bs, n_features]
```

**权重：**

```
W : [n_features, out_dim]
```

**输出：**

$$
Y = XW + b
$$

特点：

* 整个输入是一个完整的特征向量
* 只做一次线性变换
* 不涉及空间或序列结构
* 不需要滑动窗口

---

# 2. CNN：在空间维上重复做局部线性变换

**输入：**

```
X : [bs, c_in, h, w]
```

**卷积核：**

```
K : [c_out, c_in, k, k]
```

卷积是：

> 对每个位置 (i,j)，从输入取一个局部 patch `[c_in, k, k]`，展平为向量后，乘以权重矩阵 `W`。

数学表达（单通道示例）：

$$
Y_{i,j} = \sum_{u=0}^{k-1} \sum_{v=0}^{k-1} X_{i+u, j+v} \cdot K_{u,v}
$$

向量化（patch → vec）后：

$$
Y_{i,j} = \text{vec}(X_{i:i+k,j:j+k}) \cdot \text{vec}(K)
$$

矩阵实现（im2col）：

```
[bs, c_in, h, w]
    → patches : [bs, hw_out, c_in*k*k]
    → matmul  : [bs, hw_out, c_out]
    → reshape : [bs, c_out, h_out, w_out]
```

**为什么比 MLP 多一步？**
因为 CNN 需要“滑动窗口”去获得局部 patch，而 MLP 不处理空间结构。

---

# 3. RNN：在时间维上重复做线性变换

**输入：**

```
X : [T, bs, d_in]
```

RNN 单步公式：

$$
h_t = \phi ( X_t W_{xh} + h_{t-1} W_{hh} + b_h )
$$

按时间拆开：

```python
for t in range(T):
    x_t = X[t]            # [bs, d_in]
    h_t = x_t @ W_xh + h_{t-1} @ W_hh
```

输出层：

$$
o_t = h_t W_{hy} + b_y
$$

本质：

* 与 MLP 相同，输入都是 `[bs, d_in]`
* RNN 只是在 **时间维 T 上重复** 应用线性变换
* 同一套参数在每个时间步共享

---

# 4. 各网络的“重复作用维度”对比

所有深度学习网络都可以抽象为：

> **把输入拆成多个向量，对每个向量应用同一个线性变换 W**

| 模型类型       | 输入形状            | 重复操作的维度                 | 小向量是什么？        |
| ---------- | --------------- | ----------------------- | -------------- |
| 线性回归 / MLP | `[bs, n]`       | batch (`bs`)            | 整个特征向量         |
| CNN        | `[bs, c, h, w]` | batch × 空间 (`bs * h*w`) | 每个像素的感受野 patch |
| RNN        | `[T, bs, d]`    | batch × 时间 (`T * bs`)   | 每个时间步的输入向量     |
